In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgbm

In [2]:
# =============================================================================
# VERİLERİ YÜKLEME
# =============================================================================

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('testFeatures.csv')

print(f"Eğitim verisi satır sayısı: {train_df.shape[0]}")
print(f"Test verisi satır sayısı: {test_df.shape[0]}")

Eğitim verisi satır sayısı: 227520
Test verisi satır sayısı: 45504


In [3]:
# =============================================================================
# TARİH ÖZELLİKLERİNİ OLUŞTURMA
# =============================================================================

def tarih_ozelliklerini_ekle(veri):
    veri['tarih'] = pd.to_datetime(veri['tarih'])

    veri['ay'] = veri['tarih'].dt.month
    veri['gun'] = veri['tarih'].dt.day
    veri['haftanin_gunu'] = veri['tarih'].dt.dayofweek
    veri['yil'] = veri['tarih'].dt.year
    
    return veri

train_df = tarih_ozelliklerini_ekle(train_df)
test_df = tarih_ozelliklerini_ekle(test_df)

In [4]:
# =============================================================================
# KATEGORİK DEĞİŞKENLERİ SAYISAL FORMATA ÇEVİRME
# =============================================================================

kategorik_degiskenler = ['ürün', 'ürün kategorisi', 'ürün üretim yeri', 'market', 'şehir']

for degisken in kategorik_degiskenler:
    encoder = LabelEncoder()
    train_df[f'{degisken}_kod'] = encoder.fit_transform(train_df[degisken])
    
    yeni_kategoriler = set(test_df[degisken].unique()) - set(encoder.classes_)
    if yeni_kategoriler:
        print(f"'{degisken}' için {len(yeni_kategoriler)} yeni kategori bulundu")
        en_sik_kategori = train_df[degisken].value_counts().index[0]
        test_df[degisken] = test_df[degisken].apply(
            lambda x: en_sik_kategori if x in yeni_kategoriler else x
        )
    
    test_df[f'{degisken}_kod'] = encoder.transform(test_df[degisken])

In [5]:
# =============================================================================
# GRUP İSTATİSTİKLERİNİ OLUŞTURMA
# =============================================================================

urun_fiyat_ortalama = train_df.groupby('ürün_kod')['ürün fiyatı'].agg([
    'mean', 'median', 'std'
]).reset_index()
urun_fiyat_ortalama.columns = ['ürün_kod', 'urun_ort_fiyat', 'urun_medyan_fiyat', 'urun_std_fiyat']

kategori_fiyat_ortalama = train_df.groupby('ürün kategorisi_kod')['ürün fiyatı'].agg([
    'mean'
]).reset_index()
kategori_fiyat_ortalama.columns = ['ürün kategorisi_kod', 'kategori_ort_fiyat']

market_fiyat_ortalama = train_df.groupby('market_kod')['ürün fiyatı'].agg([
    'mean'
]).reset_index()
market_fiyat_ortalama.columns = ['market_kod', 'market_ort_fiyat']

train_df = train_df.merge(urun_fiyat_ortalama, on='ürün_kod', how='left')
test_df = test_df.merge(urun_fiyat_ortalama, on='ürün_kod', how='left')

train_df = train_df.merge(kategori_fiyat_ortalama, on='ürün kategorisi_kod', how='left')
test_df = test_df.merge(kategori_fiyat_ortalama, on='ürün kategorisi_kod', how='left')

train_df = train_df.merge(market_fiyat_ortalama, on='market_kod', how='left')
test_df = test_df.merge(market_fiyat_ortalama, on='market_kod', how='left')

for veri in [train_df, test_df]:
    for sutun in veri.columns:
        if 'fiyat' in sutun and veri[sutun].isnull().sum() > 0:
            veri[sutun].fillna(veri[sutun].mean(), inplace=True)

In [6]:
# =============================================================================
# ÖZELLİKLERİ HAZIRLAMA
# =============================================================================

kullanilacak_ozellikler = [
    'ürün_kod', 'ürün kategorisi_kod', 'ürün üretim yeri_kod', 
    'market_kod', 'şehir_kod', 'ürün besin değeri',
    'ay', 'gun', 'haftanin_gunu', 'yil',
    'urun_ort_fiyat', 'urun_medyan_fiyat', 'urun_std_fiyat',
    'kategori_ort_fiyat', 'market_ort_fiyat'
]

mevcut_ozellikler = [ozellik for ozellik in kullanilacak_ozellikler 
                     if ozellik in train_df.columns]
print(f"Kullanılacak özellik sayısı: {len(mevcut_ozellikler)}")

X = train_df[mevcut_ozellikler]
y = train_df['ürün fiyatı']
X_test = test_df[mevcut_ozellikler]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train_scaled, X_val_scaled, _, _ = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

Kullanılacak özellik sayısı: 15


In [7]:
# =============================================================================
# MODELLERİ EĞİTME VE KARŞILAŞTIRMA
# =============================================================================

model_sonuclari = {}

# 1. Decision Tree
print("\n>> Decision Tree Modeli Eğitiliyor...")
decision_tree = DecisionTreeRegressor(max_depth=15, random_state=42)
decision_tree.fit(X_train, y_train)
y_pred_dt = decision_tree.predict(X_val)
rmse_dt = np.sqrt(mean_squared_error(y_val, y_pred_dt))
print(f"Decision Tree RMSE: {rmse_dt:.4f}")
model_sonuclari['Decision Tree'] = {'model': decision_tree, 'rmse': rmse_dt}

# 2. Random Forest
print("\n>> Random Forest Modeli Eğitiliyor...")
random_forest = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42)
random_forest.fit(X_train, y_train)
y_pred_rf = random_forest.predict(X_val)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
print(f"Random Forest RMSE: {rmse_rf:.4f}")
model_sonuclari['Random Forest'] = {'model': random_forest, 'rmse': rmse_rf}

# 3. LightGBM
print("\n>> LightGBM Modeli Eğitiliyor...")
lgbm_model = lgbm.LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42)
lgbm_model.fit(X_train, y_train)
y_pred_lgbm = lgbm_model.predict(X_val)
rmse_lgbm = np.sqrt(mean_squared_error(y_val, y_pred_lgbm))
print(f"LightGBM RMSE: {rmse_lgbm:.4f}")
model_sonuclari['LightGBM'] = {'model': lgbm_model, 'rmse': rmse_lgbm}

# 4. Neural Network
print("\n>> Neural Network Modeli Eğitiliyor...")
mlp_model = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    max_iter=250, 
    random_state=42
)
mlp_model.fit(X_train_scaled, y_train)
y_pred_mlp = mlp_model.predict(X_val_scaled)
rmse_mlp = np.sqrt(mean_squared_error(y_val, y_pred_mlp))
print(f"Neural Network RMSE: {rmse_mlp:.4f}")
model_sonuclari['Neural Network'] = {'model': mlp_model, 'rmse': rmse_mlp}



>> Decision Tree Modeli Eğitiliyor...
Decision Tree RMSE: 1.6444

>> Random Forest Modeli Eğitiliyor...
Random Forest RMSE: 1.3660

>> LightGBM Modeli Eğitiliyor...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009483 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 429
[LightGBM] [Info] Number of data points in the train set: 182016, number of used features: 14
[LightGBM] [Info] Start training from score 17.023668
LightGBM RMSE: 1.2424

>> Neural Network Modeli Eğitiliyor...


c:\Users\hp\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural Network RMSE: 1.0850


In [8]:
# =============================================================================
# EN İYİ MODELİ SEÇME VE TAHMİN YAPMA
# =============================================================================

best_model_name = min(model_sonuclari.items(), key=lambda x: x[1]['rmse'])[0]
best_model = model_sonuclari[best_model_name]['model']
best_rmse = model_sonuclari[best_model_name]['rmse']

print(f"\nEN İYİ MODEL: {best_model_name}")
print(f"RMSE: {best_rmse:.4f}")

if best_model_name == 'Neural Network':
    test_predictions = best_model.predict(X_test_scaled)
else:
    test_predictions = best_model.predict(X_test)

test_predictions = np.maximum(0, test_predictions)


EN İYİ MODEL: Neural Network
RMSE: 1.0850


In [9]:
# =============================================================================
# TAHMİNLERİ KAYDETME 
# =============================================================================

current_time = datetime.datetime.now()
timestamp = current_time.strftime("%Y%m%d_%H%M%S")  # YYYYMMDD_HHMMSS formatı

submission_filename = f'submission_{timestamp}.csv'

submission = pd.DataFrame({
    'id': test_df['id'],
    'ürün fiyatı': test_predictions
})

submission.to_csv(submission_filename, index=False)
print(f"Tahminler '{submission_filename}' dosyasına kaydedildi!")

Tahminler 'submission_20250621_203443.csv' dosyasına kaydedildi!


In [10]:
# =============================================================================
# ÖZET İSTATİSTİKLER
# =============================================================================

print(f"Tahmin Edilen Minimum Fiyat: {test_predictions.min():.2f}")
print(f"Tahmin Edilen Maksimum Fiyat: {test_predictions.max():.2f}")
print(f"Tahmin Edilen Ortalama Fiyat: {test_predictions.mean():.2f}")

print("\n=== Model Performans Karşılaştırması ===")
for model_adi, sonuc in model_sonuclari.items():
    print(f"{model_adi:<20} RMSE: {sonuc['rmse']:.4f}")

Tahmin Edilen Minimum Fiyat: 3.75
Tahmin Edilen Maksimum Fiyat: 197.67
Tahmin Edilen Ortalama Fiyat: 46.59

=== Model Performans Karşılaştırması ===
Decision Tree        RMSE: 1.6444
Random Forest        RMSE: 1.3660
LightGBM             RMSE: 1.2424
Neural Network       RMSE: 1.0850
